# High-level Keras (Theano) Example

In [1]:
import os
import sys
import numpy as np
os.environ['KERAS_BACKEND'] = "theano"
import keras as K
import theano
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from common.params import *
from common.utils import *

Using Theano backend.
Using cuDNN version 6021 on context None
Mapped name None to device cuda: Tesla K80 (BF4E:00:00.0)


In [2]:
# channels_first is faster
K.backend.set_image_data_format('channels_first')

In [2]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Keras: ", K.__version__)
print("Numpy: ", np.__version__)
print("Theano: ", theano.__version__)
print(K.backend.backend())
# Should be channels-first, otherwise slow
print(K.backend.image_data_format())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Keras:  2.0.8
Numpy:  1.13.1
Theano:  0.10.0beta1.dev-RELEASE
theano
channels_first


In [3]:
#CuDNN auto-tune
theano.config.dnn.conv.algo_fwd = "time_once"
theano.config.dnn.conv.algo_bwd_filter = "time_once"
theano.config.dnn.conv.algo_bwd_data = "time_once"

In [4]:
def create_symbol():
    model = Sequential()
    
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(3, 32, 32)))
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
        
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(N_CLASSES, activation='softmax'))
    return model

In [5]:
def init_model(m):
    m.compile(
        loss = "categorical_crossentropy",
        optimizer = K.optimizers.SGD(LR, MOMENTUM),
        metrics = ['accuracy'])
    return m

In [6]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = cifar_for_library(channel_first=True, one_hot=True)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Done.
Extracting files...
Done.
Preparing train set...
Preparing test set...
Done.
(50000, 3, 32, 32) (10000, 3, 32, 32) (50000, 10) (10000, 10)
float32 float32 int32 int32
CPU times: user 2.92 s, sys: 1.43 s, total: 4.34 s
Wall time: 27.7 s


In [7]:
%%time
# Load symbol
sym = create_symbol()

CPU times: user 964 ms, sys: 116 ms, total: 1.08 s
Wall time: 3.16 s


In [8]:
%%time
# Initialise model
model = init_model(sym)

CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 11.3 ms


In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 50, 32, 32)        1400      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 32, 32)        22550     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 16, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 16, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 100, 16, 16)       45100     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 100, 16, 16)       90100     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 100, 8, 8)         0         
__________

In [10]:
%%time
# Train model
model.fit(x_train,
          y_train,
          batch_size=BATCHSIZE,
          epochs=EPOCHS,
          verbose=1)

Epoch 1/10
50000/50000 [==============================] - 22s - loss: 1.8528 - acc: 0.3227    
Epoch 2/10
50000/50000 [==============================] - 25s - loss: 1.4089 - acc: 0.4873    
Epoch 3/10
50000/50000 [==============================] - 25s - loss: 1.1726 - acc: 0.5821    
Epoch 4/10
50000/50000 [==============================] - 25s - loss: 1.0034 - acc: 0.6460    
Epoch 5/10
50000/50000 [==============================] - 26s - loss: 0.8888 - acc: 0.6873    
Epoch 6/10
50000/50000 [==============================] - 26s - loss: 0.7867 - acc: 0.7238    
Epoch 7/10
50000/50000 [==============================] - 26s - loss: 0.7200 - acc: 0.7473    
Epoch 8/10
50000/50000 [==============================] - 26s - loss: 0.6515 - acc: 0.7722    
Epoch 9/10
50000/50000 [==============================] - 27s - loss: 0.5975 - acc: 0.7897    - ETA: 1s - 
Epoch 10/10
50000/50000 [==============================] - 27s - loss: 0.5483 - acc: 0.8055    
CPU times: user 2min 25s, sys: 51.5 s

In [11]:
%%time
y_guess = model.predict(x_test, batch_size=BATCHSIZE)
y_guess = np.argmax(y_guess, axis=-1)
y_truth = np.argmax(y_test, axis=-1)

CPU times: user 6.33 s, sys: 408 ms, total: 6.74 s
Wall time: 8.27 s


In [12]:
print("Accuracy: ", sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.7783
